In [5]:
#!pip install seaborn pandas kaggle

In [6]:
#from google.colab import files
#!mkdir ~/.kaggle
#!cp kaggle.json ~/.kaggle/
#!chmod 600 ~/.kaggle/kaggle.json

In [7]:
#!kaggle datasets download -d imsparsh/deam-mediaeval-dataset-emotional-analysis-in-music

In [8]:
#!unzip 'deam-mediaeval-dataset-emotional-analysis-in-music.zip'
#%rm 'deam-mediaeval-dataset-emotional-analysis-in-music.zip'

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import glob
%matplotlib inline

In [125]:
df_annotations_1_2000 = pd.read_csv('/content/DEAM_Annotations/annotations/annotations averaged per song/song_level/static_annotations_averaged_songs_1_2000.csv')
df_annotations_2000_2058 = pd.read_csv('/content/DEAM_Annotations/annotations/annotations averaged per song/song_level/static_annotations_averaged_songs_2000_2058.csv')

In [126]:
df_annotations_1_2000.drop([' valence_std',' arousal_std'], axis=1, inplace=True)
df_annotations_2000_2058.drop(df_annotations_2000_2058.columns.difference(['song_id',' valence_mean', ' arousal_mean']), axis=1, inplace=True)
df_annotations = pd.concat([df_annotations_1_2000, df_annotations_2000_2058])
df_annotations.set_index('song_id', inplace=True)

In [127]:
emotions = {
    0: 'pleased',
    1: 'happy',
    2: 'excited',
    3: 'anger',
    4: 'anger',
    5: 'anger',
    6: 'sad',
    7: 'bored',
    8: 'sleepy',
    9: 'contentment',
    10: 'contentment',
    11: 'contentment'
}

In [128]:
def normilize(x):
    return ((x-5)/4)

def get_class(arousal, valence):
  angle = np.arctan2(normilize(arousal), normilize(valence))
  angle = (angle + 2 * np.pi) % (2 * np.pi)
  num_emotions = 12
  emotion_class = int(np.floor(angle / (2 * np.pi / num_emotions)))
  return emotions[emotion_class]

In [129]:
df_features = pd.read_csv(glob.glob('/content/features/features/2.csv')[0], delimiter=';')
df_features.drop(['frameTime'], axis=1, inplace=True)
df = pd.DataFrame(columns = df_features.columns)
df['emotion_class'] = ''

for id in df_annotations.index:
  df_features = pd.read_csv(glob.glob('/content/features/features/' + str(id) + '.csv')[0], delimiter=';')
  df_features.drop(['frameTime'], axis=1, inplace=True)

  new_features = []
  for col in df_features.columns:
    new_features.append(np.mean(df_features[col]))
  new_features.append(get_class(df_annotations.loc[id][' arousal_mean'], df_annotations.loc[id][' valence_mean']))

  df.loc[len(df.index)] = new_features

df.set_index(df_annotations.index, inplace=True)

In [130]:
len(df['emotion_class'].unique())

8

In [131]:
df

,F0final_sma_stddev,F0final_sma_amean,voicingFinalUnclipped_sma_stddev,voicingFinalUnclipped_sma_amean,jitterLocal_sma_stddev,jitterLocal_sma_amean,jitterDDP_sma_stddev,jitterDDP_sma_amean,shimmerLocal_sma_stddev,shimmerLocal_sma_amean,...,pcm_fftMag_mfcc_sma_de[10]_amean,pcm_fftMag_mfcc_sma_de[11]_stddev,pcm_fftMag_mfcc_sma_de[11]_amean,pcm_fftMag_mfcc_sma_de[12]_stddev,pcm_fftMag_mfcc_sma_de[12]_amean,pcm_fftMag_mfcc_sma_de[13]_stddev,pcm_fftMag_mfcc_sma_de[13]_amean,pcm_fftMag_mfcc_sma_de[14]_stddev,pcm_fftMag_mfcc_sma_de[14]_amean,emotion_class
song_id,,,,,,,,,,,,,,,,,,,,,
2,19.860414,93.884056,0.027706,0.742852,0.117239,0.099609,0.112936,0.095736,0.118050,0.180007,...,5.369898e-04,2.230241,2.795768e-04,1.982235,2.542186e-04,1.918530,1.726519e-04,1.694786,-3.051494e-05,bored
3,19.000312,62.682589,0.031728,0.754430,0.083326,0.056241,0.076878,0.054784,0.133759,0.147028,...,2.858169e-04,1.792956,2.030164e-04,1.709216,5.744590e-05,1.519361,-5.567165e-05,1.391123,-8.473427e-06,bored
4,48.175781,92.850316,0.042853,0.753095,0.136936,0.081527,0.153833,0.095950,0.136401,0.161130,...,1.650659e-05,2.083527,4.228550e-04,2.022455,2.467478e-04,1.852797,3.383194e-05,1.590440,2.433487e-05,happy
5,61.043677,158.673853,0.032721,0.757328,0.117823,0.101659,0.135685,0.108718,0.101054,0.189116,...,-2.482530e-09,2.125705,-1.203645e-09,1.899873,-7.903614e-10,1.758863,2.250000e-09,1.553451,-7.647590e-10,anger
7,14.317295,83.823484,0.036337,0.787512,0.098302,0.059757,0.096449,0.060557,0.169927,0.163200,...,-7.397941e-04,1.652931,-5.146167e-04,1.616115,-3.012753e-04,1.455275,5.290876e-04,1.382201,2.044570e-04,excited
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,60.439987,135.227449,0.033471,0.794851,0.112610,0.074239,0.133267,0.083880,0.117681,0.163266,...,1.354159e-03,1.848548,2.334927e-03,1.769273,1.942244e-03,1.523673,3.964342e-04,1.475354,-4.890418e-04,contentment
2055,99.440455,132.265650,0.059393,0.757890,0.105004,0.066159,0.111205,0.071426,0.117365,0.138263,...,-2.726037e-04,2.408328,4.060140e-04,2.307614,6.132043e-04,2.076692,7.187288e-04,1.909902,4.615625e-04,anger
2056,53.946587,109.565479,0.061583,0.758426,0.097751,0.068848,0.100636,0.069814,0.139728,0.175065,...,5.606182e-04,2.241761,-4.000093e-04,2.108922,-1.759559e-04,1.997892,3.087410e-04,1.819372,4.370266e-04,contentment


In [132]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

df['emotion_class']= label_encoder.fit_transform(df['emotion_class'])

In [133]:
print(df['emotion_class'].unique())

[1 4 0 3 6 7 2 5]


In [134]:
from sklearn.model_selection import train_test_split
import re

regex = re.compile(r"\[|\]|<", re.IGNORECASE)
df.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in df.columns.values]

X = df.drop('emotion_class', axis=1, inplace=False)
y = df['emotion_class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [135]:
from sklearn.preprocessing import StandardScaler

normalizer = StandardScaler()

X_train = normalizer.fit_transform(X_train)
X_test = normalizer.fit_transform(X_test)

In [136]:
import xgboost as xgb

xgb.XGBClassifier().get_params()

{'objective': 'binary:logistic',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': None,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [140]:
from sklearn.metrics import accuracy_score

model_xgb = xgb.XGBClassifier(learning_rate=0.1)

model_xgb.fit(X_train, y_train, verbose=True)

pred = model_xgb.predict(X_test)
print("Accuracy: "+str(accuracy_score(y_test, pred))+'\n')

Accuracy: 0.3234750462107209



In [147]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=0)
gb.fit(X_train, y_train)

pred = gb.predict(X_test)
print("Accuracy: "+str(accuracy_score(y_test, pred)))

Accuracy: 0.2975970425138632


In [152]:
from sklearn.decomposition import PCA, FastICA

pca = PCA(0.95)
X_pca = pca.fit_transform(X)

X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.3, random_state=0)

model_xgb = xgb.XGBClassifier(learning_rate=0.1)

model_xgb.fit(X_train_pca, y_train_pca, verbose=True)

pred = model_xgb.predict(X_test_pca)
print("Accuracy: "+str(accuracy_score(y_test_pca, pred)))

Accuracy: 0.22735674676524953


In [154]:
ica = FastICA()
X_ica = ica.fit_transform(X)

X_train_ica, X_test_ica, y_train_ica, y_test_ica = train_test_split(X_ica, y, test_size=0.3, random_state=0)

model_xgb = xgb.XGBClassifier(learning_rate=0.1)

model_xgb.fit(X_train_ica, y_train_ica, verbose=True)

pred = model_xgb.predict(X_test_ica)
print("Accuracy: "+str(accuracy_score(y_test_ica, pred)))

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_fastica.py:123: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Accuracy: 0.20517560073937152
